<a href="https://colab.research.google.com/github/sumithdcosta/Python/blob/master/BlockChain/python_blockchain_training_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!mkdir utility

mkdir: cannot create directory ‘utility’: File exists


In [57]:
!pip install  pycrypto

In [58]:
!pip install flask
!pip install flask-cors
!pip install flask-ngrok

In [67]:
%%writefile utility/__init__.py
'''package'''
from utility.hash_util import hash_string_256

__all__ = ['hash_string_256']

Overwriting utility/__init__.py


In [68]:
%%writefile utility/hash_util.py
import hashlib as hl
import json

#__all__ = ['hash_string_256','hash_block']

def hash_string_256(string):
  return hl.sha256(string).hexdigest()

def hash_block(block):
  #print(block)
  hashable_block = block.__dict__.copy()
  hashable_block['transactions'] = [tx.to_ordered_dict() for tx in hashable_block['transactions']]
  return hash_string_256(json.dumps(hashable_block, sort_keys=True).encode())


Overwriting utility/hash_util.py


In [69]:
%%writefile utility/verification.py
""" Provides verification helper methods."""

from utility.hash_util import hash_string_256, hash_block
from wallet import Wallet

class Verification:
  """A helper class which offer various static and class-based veriffication"""
  @staticmethod
  def valid_proof(transactions, last_hash, proof):
    guess = (str([tx.to_ordered_dict() for tx in transactions]) + str(last_hash) + str(proof)).encode()
    guess_hash = hash_string_256(guess)
    #print(guess_hash)
    return guess_hash[0:2] == '00'

  @classmethod
  def verify_chain(cls, blockchain):
    """verify current blockchain"""
    for (index, block) in enumerate(blockchain):
      if index == 0:
        continue
      if block.previous_hash != hash_block(blockchain[index -1]):
        return False
      if not cls.valid_proof(block.transactions[:-1], block.previous_hash, block.proof):
        print('Prood of work invalid')
        return False
    return True

  @staticmethod
  def verify_transaction(transaction, get_balance, check_funds=True):
    if check_funds:
      sender_balance = get_balance()
      return sender_balance >= transaction.amount and Wallet.verify_transaction(transaction)
    else:
      return Wallet.verify_transaction(transaction)

  @classmethod
  def verify_transactions(cls, open_transactions, get_balance):
    return all([cls.verify_transaction(tx, get_balance, check_funds=False) for tx in open_transactions])


Overwriting utility/verification.py


In [70]:
%%writefile utility/printable.py
class Printable:
  def __repr__(self):
    return str(self.__dict__)


Overwriting utility/printable.py


In [71]:
%%writefile block.py
from time import time
from utility.printable import Printable

class Block(Printable):
  def __init__(self, index, previous_hash, transactions, proof, time=time()):
    self.index = index
    self.previous_hash = previous_hash
    self.transactions = transactions
    self.proof = proof
    self.time = time


Overwriting block.py


In [72]:
%%writefile transaction.py
from collections import OrderedDict
from utility.printable import Printable

class Transaction(Printable):
  def __init__(self, sender, recipient, signature, amount):
    self.sender = sender
    self.recipient = recipient
    self.signature = signature
    self.amount = amount
    



  def to_ordered_dict(self):
      return OrderedDict([('sender', self.sender),
                          ('recipient', self.recipient),
                          ('amount', self.amount)])

Overwriting transaction.py


In [87]:
%%writefile wallet.py
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import SHA256
import Crypto.Random
import binascii 
import sys

class Wallet:
  def __init__(self):
    self.private_key = None
    self.public_key = None

  def save_keys(self):
    if self.public_key != None and self.private_key != None:
      #print(self.public_key)
      #print(self.private_key)
      try : 
        with open('wallet.txt', mode='w') as f:
          f.write(self.public_key)
          f.write('\n')
          f.write(self.private_key)
        return True
      except(IOError, IndexError):
        print('Saving wallet failed ...')
        return False

  def create_keys(self):
    private_key, public_key = self.generate_keys()
    self.private_key = private_key
    self.public_key = public_key 


  def load_keys(self):
    try:
      with open('wallet.txt', mode='r') as f:
        keys = f.readlines()
        public_key = keys[0][:-1]
        private_key = keys[1]
        self.public_key = public_key
        self.private_key = private_key
      return True
    except(IOError, IndexError) as exc:
      print('Loading wallet failed ...')
      print("Oops!",exc)
      print("Oops!",sys.exc_info()[0],"occured.")
      return False


  def generate_keys(self):
    private_key = RSA.generate(1024,Crypto.Random.new().read)
    public_key = private_key.publickey()
    return (binascii.hexlify(private_key.exportKey(format='DER')).decode('ascii'),
            binascii.hexlify(public_key.exportKey(format='DER')).decode('ascii'))
    
  def sign_transaction(self, sender, recipient, amount):
    signer = PKCS1_v1_5.new(RSA.importKey(binascii.unhexlify(self.private_key)))
    h = SHA256.new((str(sender) + str(recipient) + str(amount)).encode('utf8'))
    signature = signer.sign(h)
    return binascii.hexlify(signature).decode('ascii')

  @staticmethod
  def verify_transaction(transaction):
    public_key = RSA.importKey(binascii.unhexlify(transaction.sender))
    verifier = PKCS1_v1_5.new(public_key)
    h = SHA256.new((str(transaction.sender) + str(transaction.recipient) + str(transaction.amount)).encode('utf8'))
    return verifier.verify(h, binascii.unhexlify(transaction.signature))


Overwriting wallet.py


In [88]:
%%writefile blockchain.py
# try except block : 
#####################################################
import functools
import hashlib as hl
import json
from collections import OrderedDict
import sys

from utility.hash_util import  hash_block
from block import Block
from transaction import Transaction
from time import time
from utility.verification import Verification
from wallet import Wallet

#global variable
MINING_REWARD = 10

#print(__name__)

class Blockchain:
  def __init__(self, hosting_node_id):
    genesis_block = Block(index = 0,
                        previous_hash= '',
                        transactions = [],
                        proof = 100,
                        time = 0)

    self.chain = [genesis_block]
    self._open_transactions = []
    self.hosting_node = hosting_node_id
    owner = 'Sumith'
    participants = {owner}
    self.load_data()

  @property
  def chain(self):
      return self._chain[:]
  
  @chain.setter
  def chain(self, val):
    self._chain = val

  def get_open_transactions(self):
    return self._open_transactions[:]

  def load_data(self):
    try:
      with open('blockchain.txt', mode='r') as f:
        file_content = f.readlines()

        blockchain = json.loads(file_content[0][:-1])
        updated_blockchain=[]
        for block in blockchain:
          converted_tx = [Transaction(tx['sender'],
                                      tx['recipient'],
                                      tx['signature'],
                                      tx['amount']) for tx in block['transactions']]
          updated_block = Block(block['index'],
                                block['previous_hash'],
                                converted_tx,
                                block['proof'],
                                block['time'])
          updated_blockchain.append(updated_block)
        self.chain = updated_blockchain
      
        open_transactions = json.loads(file_content[1])
        updated_transactions = []
        for tx in open_transactions:
          updated_transaction = Transaction(tx['sender'],
                                             tx['recipient'],
                                            tx['signature'],
                                             tx['amount'])
          updated_transactions.append(updated_transaction) 
        self._open_transactions = updated_transactions
    except (IOError, IndexError):                    
      pass
    except ValueError:
      print('Value error!')
    except Exception as exc:
      print("Oops!",exc)
      print("Oops!",sys.exc_info()[0],"occured.")
      sys.exit(1)
    finally:
      print('Cleanup!')

  def save_data(self):
    try :
      with open('blockchain.txt', mode='w') as f:
        saveable_chain = [block.__dict__ for block in [Block(
            block_el.index,
            block_el.previous_hash,
            [tx.__dict__ for tx in block_el.transactions],
            block_el.proof,
            block_el.time
            ) for block_el in self._chain]]

        f.write(json.dumps(saveable_chain))
        f.write('\n')
        saveable_tx = [tx.__dict__ for tx in self._open_transactions]
        f.write(json.dumps(saveable_tx))
    except IOError:
      print('Error saving file!')

  def proof_of_work(self):
    last_block = self._chain[-1]
    last_hash = hash_block(last_block)
    proof = 0
    while not Verification.valid_proof(self._open_transactions, last_hash, proof):
      proof += 1
    return proof

  def get_balance(self):
    if self.hosting_node == None:
      return None
    participant = self.hosting_node
    tx_sender = [[tx.amount for tx in block.transactions if tx.sender == participant] for block in self._chain]
    open_tx_sender = [tx.amount for tx in self._open_transactions if tx.sender == participant]
    tx_sender.append(open_tx_sender)
    amount_sent = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_sender,0)
    tx_recipient = [[tx.amount for tx in block.transactions if tx.recipient == participant] for block in self._chain]
    amount_received = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_recipient,0)
    return  amount_received - amount_sent


  def get_last_blockchain_value(self):
    """ returns the last value of the current bloackchain """
    #function to return the value
    if len(self._chain) < 1 :
      return None
    return self._chain[-1]

  def add_transaction(self, recipient,sender, signature, amount=1.0):
    if self.hosting_node == None:
      return False
    transaction = Transaction(sender, recipient, signature, amount)
    if Verification.verify_transaction(transaction, self.get_balance):
      self._open_transactions.append(transaction)
      self.save_data()
      return True
    return False

  def mine_block(self):
    if self.hosting_node == None:
      return None
    last_block = self._chain[-1]
    hashed_block = hash_block(last_block)
    proof = self.proof_of_work() 
    reward_transaction = Transaction(sender = 'MINING', recipient = self.hosting_node, signature = '', amount = MINING_REWARD)
    copied_transactions = self._open_transactions[:]
    for tx in copied_transactions:
      if not Wallet.verify_transaction(tx):
        return None
    copied_transactions.append(reward_transaction)
    block = Block(index = len(self._chain),
                        previous_hash = hashed_block,
                        transactions = copied_transactions,
                        proof = proof,
                        time = 0)
    self._chain.append(block)
    self._open_transactions = []
    self.save_data()
    return block




Overwriting blockchain.py


In [89]:
%%writefile node.py
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify
from flask_cors import CORS

from wallet import Wallet
from blockchain import Blockchain

app = Flask(__name__)
wallet = Wallet()
blockchain = Blockchain(wallet.public_key)
CORS(app)
run_with_ngrok(app)

@app.route('/wallet', methods=['POST'])
def create_keys():
  wallet.create_keys()
  if wallet.save_keys():
    global blockchain
    blockchain = Blockchain(wallet.public_key)
    response= {
      'public_key': wallet.public_key,
      'private_key': wallet.private_key
      'funds': blockchain.get_balance()
    } 
    return jsonify(response), 201
  else:
    response = {
      'message': 'saving the keys failed!'
    }
    return jsonify(response), 500
  
@app.route('/wallet', methods =['GET'])
def load_keys():
  if wallet.load_keys():
    global blockchain
    blockchain = Blockchain(wallet.public_key)
    response= {
      'public_key': wallet.public_key,
      'private_key': wallet.private_key
      'funds': blockchain.get_balance()
    } 
    global blockchain
    blockchain = Blockchain(wallet.public_key)
    return jsonify(response), 201
  else:
    response = {
      'message': 'loading the keys failed!'
    }
    return jsonify(response), 500

@app.route('/balance', methods=['GET'])
def get_balance():
  balance = blockchain.get_balance()
  if balance != None
    response = {
      'message': 'Fetched balance successfully.',
      'funds': balance
    }
  else:
    response = {
      'message': 'Loading balance failed',
      'wallet_set_up': wallet.public_key != None
    }
    return jsonify(response), 500

@app.route('/', methods = ['GET'])
def get_ui():
  return 'This works!'

@app.route('/mine', methods=['POST'])
def mine():
  block = blockchain.mine_block()
  if block != None:
    dict_block = block.__dict__.copy()
    dict_block['transactions'] = [tx.__dict__ for tx in dict_block['transactions']]
    response = {
      'message': 'Block added successfully.',
      'block': dict_block
      'funds': blockchain.get_balance()
    }
    return jsonify(response), 201
  else:
    response = {
      'message': 'Adding a block failed.',
      'wallet_set_up': wallet.public_key != None
    }
    return jsonify(response), 500

@app.route('/chain', methods=['GET'])
def get_chain():
  chain_snapshot = blockchain.chain
  dict_chain = [block.__dict__.copy() for block in chain_snapshot ]
  for dict_block in dict_chain:
    dict_block['transactions'] = [tx.__dict__ for tx in dict_block['transactions']]
  return jsonify(dict_chain), 200


if __name__ == '__main__':
  app.run()


Overwriting node.py


In [0]:
!python node.py

Cleanup!
 * Serving Flask app "node" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://ab874a36.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Cleanup!
127.0.0.1 - - [16/Mar/2020 14:44:32] "GET /wallet HTTP/1.1" 201 -
127.0.0.1 - - [16/Mar/2020 14:45:07] "POST /mine HTTP/1.1" 201 -


In [39]:
%%writefile old_node.py
from uuid import uuid4
from blockchain import Blockchain
from utility.verification import Verification
from wallet import Wallet


class Node:
  def __init__(self):
    self.wallet = Wallet()
    #self.id = str(uuid4())
    self.wallet.create_keys()
    self.blockchain = Blockchain(self.wallet.public_key)
    

  #using function for reusable code.
  def get_transaction_value(self):
    """ Retruns the input of the user(a new transaction amount) as a float
    """
    tx_recipient = input('Enter the recipent of the transaction: ') 
    tx_amount = float(input('Your trasaction amount please: ')) 
    return (tx_recipient, tx_amount)
  
  def get_user_choice(self):
    user_input = input('Your Choice : ')
    return user_input

  def print_blockchain_elements(self):
    #use for loop to output the blockchain list console
    for block in self.blockchain.chain:
      print('outputting block : ', block)
    else:
      print('-' * 20)

  def listen_for_input(self):
    waiting_for_input = True
    while waiting_for_input:
      print('Please choose')
      print('1: Add a new transaction value')
      print('2: Mine a new block')
      print('3: Output the blockchain blocks')
      print('4: check transaction validity')
      print('5: Create wallet')
      print('6: load wallet')
      print('7: Save keys')
      print('q : Quit ')
      user_choice = self.get_user_choice()
      if user_choice == '1':
        tx_data = self.get_transaction_value()
        recipient, amount = tx_data
        signature = self.wallet.sign_transaction(self.wallet.public_key, recipient, amount)
        if self.blockchain.add_transaction(recipient, self.wallet.public_key, signature,amount=amount):
          print('Added transaction')
        else:
          print('Tranaction failed')
        print(self.blockchain.get_open_transactions())
      elif user_choice == '2':
        if not self.blockchain.mine_block():
          print('Mining failed, got no wallet?')
      elif user_choice == '3':
          self.print_blockchain_elements()
      elif user_choice == '4':
        if Verification.verify_transactions(self.blockchain.get_open_transactions(), self.blockchain.get_balance):
          print("All Transactions are valid")
        else:
          print('There are invalid transaction')
      elif user_choice == '5':
        self.wallet.create_keys()
        self.blockchain = Blockchain(self.wallet.public_key)
      elif user_choice == '6':
        self.wallet.load_keys()
        self.blockchain = Blockchain(self.wallet.public_key)
      elif user_choice == '7':
        self.wallet.save_keys()
      elif user_choice == 'q':
        waiting_for_input = False
      else:
        print('Input was invalid, please pick a value from the list!')    
      if not Verification.verify_chain(self.blockchain.chain):
        self.print_blockchain_elements()
        print('Invalid block chain!')
        break
      print('Balance of {} : {:6.2f}'.format(self.wallet.public_key, self.blockchain.get_balance()))  
      print('Choice registered !')
    else:
      print('User left')
    print('Done!') 

if __name__ == '__main__':
  node = Node()
  node.listen_for_input() 

#print(__name__)


Overwriting old_node.py


In [42]:
%%writefile old_blockchain.py
# try except block : 
#####################################################
import functools
import hashlib as hl
import json
from collections import OrderedDict
import sys

from utility.hash_util import  hash_block
from block import Block
from transaction import Transaction
from time import time
from utility.verification import Verification
from wallet import Wallet

#global variable
MINING_REWARD = 10

#print(__name__)

class Blockchain:
  def __init__(self, hosting_node_id):
    genesis_block = Block(index = 0,
                        previous_hash= '',
                        transactions = [],
                        proof = 100,
                        time = 0)

    self.chain = [genesis_block]
    self._open_transactions = []
    self.hosting_node = hosting_node_id
    owner = 'Sumith'
    participants = {owner}
    self.load_data()

  @property
  def chain(self):
      return self._chain[:]
  
  @chain.setter
  def chain(self, val):
    self._chain = val

  def get_open_transactions(self):
    return self._open_transactions[:]

  def load_data(self):
    try:
      with open('blockchain.txt', mode='r') as f:
        file_content = f.readlines()

        blockchain = json.loads(file_content[0][:-1])
        updated_blockchain=[]
        for block in blockchain:
          converted_tx = [Transaction(tx['sender'],
                                      tx['recipient'],
                                      tx['signature'],
                                      tx['amount']) for tx in block['transactions']]
          updated_block = Block(block['index'],
                                block['previous_hash'],
                                converted_tx,
                                block['proof'],
                                block['time'])
          updated_blockchain.append(updated_block)
        self.chain = updated_blockchain
      
        open_transactions = json.loads(file_content[1])
        updated_transactions = []
        for tx in open_transactions:
          updated_transaction = Transaction(tx['sender'],
                                             tx['recipient'],
                                            tx['signature'],
                                             tx['amount'])
          updated_transactions.append(updated_transaction) 
        self._open_transactions = updated_transactions
    except (IOError, IndexError):                    
      pass
    except ValueError:
      print('Value error!')
    except Exception as exc:
      print("Oops!",exc)
      print("Oops!",sys.exc_info()[0],"occured.")
      sys.exit(1)
    finally:
      print('Cleanup!')

  def save_data(self):
    try :
      with open('blockchain.txt', mode='w') as f:
        saveable_chain = [block.__dict__ for block in [Block(
            block_el.index,
            block_el.previous_hash,
            [tx.__dict__ for tx in block_el.transactions],
            block_el.proof,
            block_el.time
            ) for block_el in self._chain]]

        f.write(json.dumps(saveable_chain))
        f.write('\n')
        saveable_tx = [tx.__dict__ for tx in self._open_transactions]
        f.write(json.dumps(saveable_tx))
    except IOError:
      print('Error saving file!')

  def proof_of_work(self):
    last_block = self._chain[-1]
    last_hash = hash_block(last_block)
    proof = 0
    while not Verification.valid_proof(self._open_transactions, last_hash, proof):
      proof += 1
    return proof

  def get_balance(self):
    participant = self.hosting_node
    tx_sender = [[tx.amount for tx in block.transactions if tx.sender == participant] for block in self._chain]
    open_tx_sender = [tx.amount for tx in self._open_transactions if tx.sender == participant]
    tx_sender.append(open_tx_sender)
    amount_sent = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_sender,0)
    tx_recipient = [[tx.amount for tx in block.transactions if tx.recipient == participant] for block in self._chain]
    amount_received = functools.reduce(lambda tx_sum, tx_amt: tx_sum + sum(tx_amt) if len(tx_amt) > 0 else tx_sum + 0, tx_recipient,0)
    return  amount_received - amount_sent


  def get_last_blockchain_value(self):
    """ returns the last value of the current bloackchain """
    #function to return the value
    if len(self._chain) < 1 :
      return None
    return self._chain[-1]

  def add_transaction(self, recipient,sender, signature, amount=1.0):
    if self.hosting_node == None:
      return False
    transaction = Transaction(sender, recipient, signature, amount)
    if Verification.verify_transaction(transaction, self.get_balance):
      self._open_transactions.append(transaction)
      self.save_data()
      return True
    return False

  def mine_block(self):
    if self.hosting_node == None:
      return False
    last_block = self._chain[-1]
    hashed_block = hash_block(last_block)
    proof = self.proof_of_work() 
    reward_transaction = Transaction(sender = 'MINING', recipient = self.hosting_node, signature = '', amount = MINING_REWARD)
    copied_transactions = self._open_transactions[:]
    for tx in copied_transactions:
      if not Wallet.verify_transaction(tx):
        return False
    copied_transactions.append(reward_transaction)
    block = Block(index = len(self._chain),
                        previous_hash = hashed_block,
                        transactions = copied_transactions,
                        proof = proof,
                        time = 0)
    self._chain.append(block)
    self._open_transactions = []
    self.save_data()
    return True




Overwriting old_blockchain.py


In [0]:
!python old_node.py

In [0]:
!python node.py

In [0]:
!cat wallet.txt

In [0]:
ls -rlt utility/

total 20
-rw-r--r-- 1 root root   90 Mar 13 05:27 __init__.py
-rw-r--r-- 1 root root  404 Mar 13 05:27 hash_util.py
-rw-r--r-- 1 root root 1219 Mar 13 05:27 verification.py
-rw-r--r-- 1 root root   68 Mar 13 05:27 printable.py
drwxr-xr-x 2 root root 4096 Mar 13 05:40 __pycache__/


In [0]:
ls -rlt

total 48
drwxr-xr-x 1 root root 4096 Mar  3 18:11 sample_data/
drwxr-xr-x 3 root root 4096 Mar 13 05:40 utility/
-rw-r--r-- 1 root root 1398 Mar 13 06:06 wallet.py
-rw-r--r-- 1 root root 3098 Mar 13 06:06 node.py
-rw-r--r-- 1 root root  307 Mar 13 06:06 block.py
-rw-r--r-- 1 root root  428 Mar 13 06:06 transaction.py
-rw-r--r-- 1 root root 5349 Mar 13 06:06 blockchain.py
drwxr-xr-x 2 root root 4096 Mar 13 06:06 __pycache__/
-rw-r--r-- 1 root root 4835 Mar 13 06:10 blockchain.txt
-rw-r--r-- 1 root root 1541 Mar 13 06:10 wallet.txt


In [0]:
!rm blockchain.txt

In [45]:
!cat blockchain.txt

cat: blockchain.txt: No such file or directory


In [0]:
2